In [6]:
!pip install pandas openpyxl python-dateutil rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 20.1 MB/s eta 0:00:00


In [136]:
import pandas as pd, re, unicodedata
from pathlib import Path

PATH = Path("NUST Bank-Product-Knowledge.xlsx")
wb   = pd.ExcelFile(PATH)                 # handles all sheets :contentReference[oaicite:2]{index=2}
S_MAIN, S_RATE = "Main", "Rate Sheet July 1 2024"
PRODUCT_SHEETS = [s for s in wb.sheet_names if s not in {S_MAIN, S_RATE}]


In [137]:
PRODUCT_SHEETS

['LCA',
 'NAA',
 'NWA',
 'PWRA',
 'RDA',
 'VPCA',
 'VP-BA',
 'VPBA',
 'NSDA',
 'PLS',
 'CDA',
 'NMA',
 'NADA',
 'NADRA',
 'NUST4Car',
 'ESFCA',
 'NFDA',
 'NSA',
 'PF',
 'NMC',
 'NMF',
 'NSF',
 'NIF',
 'NUF',
 'NFMF',
 'NFBF',
 'PMYB &ALS',
 'NRF',
 'NHF',
 'Nust Life',
 'EFU Life',
 'Jubilee Life ',
 'HOME REMITTANCE',
 'Sheet1']

#Clean Main Sheet

In [159]:
df = wb.parse("Main", header=None).fillna("")
df

,0,1,2,3,4,5
0,,NUST BANK PRODUCTS (CONVENTIONAL)\n(Click on a...,,,,
1,,Liability Products & Services,,,Consumer Products,
2,,1,NUST Asaan Account (NAA),,1,NUST4Car – NUST Bank’s Auto Finance Facility (...
3,,2,Little Champs Account,,2,Personal Finance
4,,3,NUST Sahar Accounts,,3,NUST Master Card
5,,4,NUST Waqaar Account,,4,NUST Mortgage Finance
6,,5,PakWatan Remittance Account,,SME Products,
7,,6,NUST Home Remittance,,1,NUST Sahar Finance
8,,7,RDA Digital Customer Onboarding,,2,NUST Imarat Finance
9,,8,Current Deposit Account (CDA),,3,NUST Ujala Finance


In [160]:
df = df.drop(df.columns[0], axis=1)
df = df.drop(index=df.index[0])

In [161]:
df

,1,2,3,4,5
1,Liability Products & Services,,,Consumer Products,
2,1,NUST Asaan Account (NAA),,1,NUST4Car – NUST Bank’s Auto Finance Facility (...
3,2,Little Champs Account,,2,Personal Finance
4,3,NUST Sahar Accounts,,3,NUST Master Card
5,4,NUST Waqaar Account,,4,NUST Mortgage Finance
6,5,PakWatan Remittance Account,,SME Products,
7,6,NUST Home Remittance,,1,NUST Sahar Finance
8,7,RDA Digital Customer Onboarding,,2,NUST Imarat Finance
9,8,Current Deposit Account (CDA),,3,NUST Ujala Finance
10,9,Value Plus Current Account (Individual) (VPCA),,4,NUST Flour Mill Finance


In [166]:
import pandas as pd
import re

# 0. Load your sheet (or assume df_raw is already in memory)
# df_raw = pd.read_excel("products.xlsx", header=None)

# 1. If you aren’t 100% sure it has 5 columns, force it:
df = df.iloc[:, :5].copy()

# 2. Define categories & regex for junk rows:
CATS = {
    "Liability Products & Services",
    "Consumer Products",
    "SME Products",
    "Third Party Products"
}
NUM = re.compile(r"^\d+$")

records = []
cat_left = None
cat_right = None

# 3. Scan each row, using position instead of names:
for _, row in df.iterrows():
    hL, pL = row.iloc[0], row.iloc[1]   # header & product on left block
    hR, pR = row.iloc[3], row.iloc[4]   # header & product on right block

    # update the “current” category if we see a header
    if isinstance(hL, str) and hL.strip() in CATS:
        cat_left = hL.strip()
    if isinstance(hR, str) and hR.strip() in CATS:
        cat_right = hR.strip()

    # collect from left side
    if pd.notna(pL):
        prod = str(pL).strip()
        if prod and not NUM.fullmatch(prod) and cat_left:
            records.append({"product": prod, "category": cat_left})

    # collect from right side
    if pd.notna(pR):
        prod = str(pR).strip()
        if prod and not NUM.fullmatch(prod) and cat_right:
            records.append({"product": prod, "category": cat_right})

# 4. Build tidy two-column DataFrame
tidy = pd.DataFrame(records)
tidy = tidy.dropna(subset=["category"]).reset_index(drop=True)

tidy


,product,category
0,NUST Asaan Account (NAA),Liability Products & Services
1,NUST4Car – NUST Bank’s Auto Finance Facility (...,Consumer Products
2,Little Champs Account,Liability Products & Services
3,Personal Finance,Consumer Products
4,NUST Sahar Accounts,Liability Products & Services
5,NUST Master Card,Consumer Products
6,NUST Waqaar Account,Liability Products & Services
7,NUST Mortgage Finance,Consumer Products
8,PakWatan Remittance Account,Liability Products & Services
9,NUST Home Remittance,Liability Products & Services


In [167]:
output_file = "products.json"
tidy.to_json(output_file, orient="records", indent=2)

#Clean Rates Sheet

In [168]:
df = wb.parse("Rate Sheet July 1 2024", header=None).fillna("")
df

,0,1,2,3,4,5,6,7,8,9
0,,,,,,,,,,Main
1,,,,,,,,,,
2,,Indicative Profit Rates\n (For Conventional De...,,,,,,,,
3,,,,,,,,,,
4,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
56,,,,,,Five Years,,,0.1325,
57,,,,,,,,,,
58,,,,,,FCY,USD,GBP,EUR,
59,,,,,,Savings Account,0.0025,0.0025,0,


In [176]:
import os
from openpyxl import load_workbook

def extract_text_from_sheet(sheet):
    text = []
    for row in sheet.iter_rows():
        for cell in row:
            if cell.value is not None:
                text.append(str(cell.value))
    return "\n".join(text)

def extract_all_sheets_to_txt(path, output_dir):
    wb = load_workbook(path, data_only=True)
    os.makedirs(output_dir, exist_ok=True)

    for sheet in wb.worksheets:
        text = extract_text_from_sheet(sheet)
        sheet_name = sheet.title.replace(" ", "_")
        output_path = os.path.join(output_dir, f"{sheet_name}.txt")

        with open(output_path, "w", encoding="utf-8") as f:
            f.write(text)
        print(f"Saved: {output_path}")

# Usage
xlsx_path = "NUST Bank-Product-Knowledge.xlsx"     # Replace with your file path
output_directory = "sheets_text_output"            # Directory where .txt files will be saved

extract_all_sheets_to_txt(xlsx_path, output_directory)


Saved: sheets_text_output/Main.txt
Saved: sheets_text_output/Rate_Sheet_July_1_2024.txt
Saved: sheets_text_output/LCA.txt
Saved: sheets_text_output/NAA.txt
Saved: sheets_text_output/NWA.txt
Saved: sheets_text_output/PWRA.txt
Saved: sheets_text_output/RDA.txt
Saved: sheets_text_output/VPCA.txt
Saved: sheets_text_output/VP-BA.txt
Saved: sheets_text_output/VPBA.txt
Saved: sheets_text_output/NSDA.txt
Saved: sheets_text_output/PLS.txt
Saved: sheets_text_output/CDA.txt
Saved: sheets_text_output/NMA.txt
Saved: sheets_text_output/NADA.txt
Saved: sheets_text_output/NADRA.txt
Saved: sheets_text_output/NUST4Car.txt
Saved: sheets_text_output/ESFCA.txt
Saved: sheets_text_output/NFDA.txt
Saved: sheets_text_output/NSA.txt
Saved: sheets_text_output/PF.txt
Saved: sheets_text_output/NMC.txt
Saved: sheets_text_output/NMF.txt
Saved: sheets_text_output/NSF.txt
Saved: sheets_text_output/NIF.txt
Saved: sheets_text_output/NUF.txt
Saved: sheets_text_output/NFMF.txt
Saved: sheets_text_output/NFBF.txt
Saved: she

In [114]:
def extract_catalogue(wb, sheet="Main"):
    df = wb.parse(sheet, header=None).fillna("")
    rows, current = [], None

    for _, r in df.iterrows():
        a, b, c = map(str, r[:3])          # col-0, col-1, col-2

        # ─── category row ───
        if a and not b and not c:
            current = a.strip()

        # ─── product row  ───
        elif b.isdigit() and c:
            rows.append({"product": c.strip(), "category": current})

    return pd.DataFrame(rows)


catalogue_df = extract_catalogue(wb)
print(catalogue_df.head())

                       product category
0     NUST Asaan Account (NAA)     None
1        Little Champs Account     None
2          NUST Sahar Accounts     None
3          NUST Waqaar Account     None
4  PakWatan Remittance Account     None


In [32]:
# ---------- 1 .2  RATES (accept "Deposit" too) ---------- #
PCT_RE = re.compile(r"(\d+(?:\.\d+)?)\s*%")

def extract_rates(wb, sheet=S_RATE):
    df = wb.parse(sheet, header=None).fillna("")
    rates, product = {}, None

    for row in df.itertuples(index=False):
        cells = [str(c).strip() for c in row if str(c).strip()]
        if not cells:
            continue

        label = cells[0].lower()
        is_title = (("account" in label or "deposit" in label)
                    and "profit" not in label and "payment" not in label)

        if is_title:
            product = re.sub(r"\s+(Account|Deposit).*", "", cells[0], flags=re.I).strip()
            continue

        if product:
            for c in cells:
                m = PCT_RE.search(c)
                if m:
                    rates[product] = float(m.group(1))
                    product = None
                    break
    return pd.Series(rates, name="profit_rate")

rate_series = extract_rates(wb)
print(rate_series.head())


Series([], Name: profit_rate, dtype: object)


In [33]:
rate_series

,profit_rate


In [23]:
WH_START = ("what","how","which","does","do","is","are","can","could","would","will")

def is_question(t):
    t = t.strip().lower()
    return "?" in t or t.startswith(WH_START)

def normalise(txt):
    txt = unicodedata.normalize("NFKD", str(txt))
    txt = re.sub(r"[•\u2022–-]", " ", txt)          # bullets/dashes → space
    return re.sub(r"\s{2,}", " ", txt).strip()

def extract_qa_sheet(wb, sheet):
    col = wb.parse(sheet, header=None).fillna("").iloc[:,1]   # column B
    qa, i = [], 0
    while i < len(col):
        q = normalise(col.iat[i])
        if is_question(q):
            ans = []
            for j in range(i+1, len(col)):
                txt = normalise(col.iat[j])
                if txt and is_question(txt):
                    break
                ans.append(txt)
            qa.append({"product": sheet, "question": q, "answer": " ".join(ans)})
            i = j
        else:
            i += 1
    return pd.DataFrame(qa)

qa_df = pd.concat([extract_qa_sheet(wb, s) for s in PRODUCT_SHEETS], ignore_index=True)


IndexError: single positional indexer is out-of-bounds

In [ ]:
# attach category
qa_df = qa_df.merge(catalogue_df, on="product", how="left")

# attach profit rate where available
qa_df = qa_df.merge(rate_series, left_on="product", right_index=True, how="left")


In [15]:
import unicodedata, re

def normalise(txt):
    txt = str(txt)
    txt = unicodedata.normalize("NFKD", txt)
    txt = re.sub(r"•|\u2022|–|-", " ", txt)   # bullets/dashes → space
    txt = re.sub(r"\s{2,}", " ", txt)         # collapse whitespace
    return txt.strip()

for col in ["question", "answer"]:
    qa_df[col] = qa_df[col].apply(normalise)


NameError: name 'qa_df' is not defined

In [ ]:
qa_df.to_json("nust_bank_qa.jsonl", orient="records", lines=True, force_ascii=False)


In [ ]:
wb            = pd.ExcelFile("NUST Bank-Product-Knowledge.xlsx")
catalogue_df  = extract_catalogue(wb)
rate_series   = extract_rates(wb)
product_sheets= [s for s in wb.sheet_names if s not in {"Main", rate_series.name}]
qa_frames     = [extract_qa_sheet(wb, s) for s in product_sheets]
qa_df         = (pd.concat(qa_frames)
                   .merge(catalogue_df, on="product", how="left")
                   .merge(rate_series, left_on="product", right_index=True, how="left"))
qa_df[["question","answer"]] = qa_df[["question","answer"]].applymap(normalise)
qa_df.to_json("nust_bank_qa.jsonl", orient="records", lines=True)
